In [ ]:
import logging
import sys
import time
from pathlib import Path
from typing import Any

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

print(f"📁 Корневая директория проекта: {project_root}")

In [ ]:
from tplexity.generation.prompts import SYSTEM_PROMPT_WITH_RETRIEVER

In [ ]:
from tplexity.llm_client import get_llm

print("✅ Импорты выполнены успешно")

In [ ]:
TEST_MESSAGES = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT_WITH_RETRIEVER,
    },
    {
        "role": "user",
        "content": "Привет!",
    },
]

GENERATION_PARAMS = {"temperature": 0.7, "max_tokens": 500}

print("Тестовый запрос:")
print(f"Системный промпт: {TEST_MESSAGES[0]['content']}")
print(f"Сообщение пользователя: {TEST_MESSAGES[1]['content']}")
print(f"Параметры генерации: {GENERATION_PARAMS}")

In [ ]:
async def test_provider(
    provider: str,
    messages: list[dict],
    generation_params: dict,
) -> dict[str, Any]:
    """
    Тестирует работу LLM провайдера на тестовом запросе

    Args:
        provider (str): Название провайдера
        messages (list[dict]): Список сообщений в формате OpenAI (системный промпт + сообщение пользователя)
        generation_params (dict): Параметры генерации

    Returns:
        Словарь с результатами тестирования
    """
    result = {"provider": provider, "status": "unknown", "response": None, "time": 0, "error": None}

    start_time = time.time()

    try:
        print(f"Тестирование провайдера: {provider.upper()}")

        try:
            client = get_llm(provider)
            print(f"✅ Клиент инициализирован: model={client.model}")
        except Exception as e:
            error_msg = f"Ошибка инициализации клиента: {e}"
            result["error"] = error_msg
            result["status"] = "failed"
            print(f"❌ {error_msg}")
            return result

        try:
            response = await client.generate(
                messages=messages,
                temperature=generation_params.get("temperature"),
                max_tokens=generation_params.get("max_tokens"),
            )

            result["response"] = response
            result["status"] = "success"
            result["time"] = time.time() - start_time

            print(f"✅ Успешно за {result['time']:.2f}с")
            print(f"Ответ: {response}")

        except Exception as e:
            result["status"] = "failed"
            result["error"] = str(e)
            result["time"] = time.time() - start_time
            print(f"❌ Ошибка: {e}")

    except Exception as e:
        result["status"] = "failed"
        result["error"] = f"Критическая ошибка: {e}"
        result["time"] = time.time() - start_time
        print(f"❌ Критическая ошибка: {e}")

    return result

In [ ]:
PROVIDERS = ["qwen", "yandexgpt", "chatgpt", "deepseek"]

all_results = {}

for provider in PROVIDERS:
    result = await test_provider(
        provider=provider,
        messages=TEST_MESSAGES,
        generation_params=GENERATION_PARAMS,
    )
    all_results[provider] = result
    print()

print("Тестирование завершено")